In [1]:
import pandas as pd

In [2]:
Dataset=pd.read_csv('CKD.csv')

In [3]:
Dataset.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hrmo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [4]:
Dataset.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3,0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2,0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1,0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.0,76.459948,d,1,0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.0,50.000000,c,0,0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes


In [5]:
Dataset=pd.get_dummies(Dataset,dtype=int,drop_first=True)

In [6]:
Dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [7]:
Independent=Dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
Dependent=Dataset[['classification_yes']]

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


Param_grid={'criterion':['gini', 'entropy', 'log_loss'],
            'max_features': ['sqrt','log2',None],
           
           }

grid = GridSearchCV(RandomForestClassifier(), Param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 

grid.fit(X_train, Y_train)


Fitting 5 folds for each of 9 candidates, totalling 45 fits


c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None]},
             scoring='f1_weighted', verbose=3)

In [14]:
re=grid.cv_results_

In [15]:
grid_pred=grid.predict(X_test)

In [17]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(grid_pred,Y_test)
print(cm)

[[44  1]
 [ 1 74]]


In [18]:
from sklearn.metrics import classification_report
clf_report=classification_report(grid_pred,Y_test)
print(clf_report)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        45
           1       0.99      0.99      0.99        75

    accuracy                           0.98       120
   macro avg       0.98      0.98      0.98       120
weighted avg       0.98      0.98      0.98       120



In [21]:
from sklearn.metrics import f1_score
f1_macro=f1_score(grid_pred,Y_test)
print("The f1 sore of the best parameter{}:".format(grid.best_params_),f1_score)

The f1 sore of the best parameter{'criterion': 'entropy', 'max_features': 'log2'}: <function f1_score at 0x0000024447A03E20>


In [20]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9997037037037038

In [22]:
table=pd.DataFrame.from_dict(re)
print(table)

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.284357      0.010287         0.023950        0.010718   
1       0.280318      0.010158         0.021177        0.003203   
2       0.336078      0.013754         0.022384        0.003492   
3       0.289059      0.013571         0.021442        0.003337   
4       0.286558      0.010529         0.022546        0.005964   
5       0.355176      0.016135         0.022493        0.004404   
6       0.287575      0.015678         0.021946        0.002027   
7       0.281991      0.005642         0.021861        0.002949   
8       0.254164      0.032730         0.011127        0.001580   

  param_criterion param_max_features  \
0            gini               sqrt   
1            gini               log2   
2            gini               None   
3         entropy               sqrt   
4         entropy               log2   
5         entropy               None   
6        log_loss               sqrt   
7        